<a href="https://colab.research.google.com/github/changyong93/Natural-language-processing-with-chat-bot/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%85%EB%AC%B8(11_3_RNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EB%A5%98_%EB%A1%9C%EC%9D%B4%ED%84%B0_%EB%89%B4%EC%8A%A4_%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 로이터 뉴스 분류하기(Reuters News Classification)

## 로이터 뉴스 데이터에 대한 이해

In [ ]:
from tensorflow.keras.datasets import reuters
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
(X_train, y_train), (X_test,y_test) = reuters.load_data(num_words=None, test_split=0.2)
#numwords : 이 데이터 셋에서 등장 빈도 순위로 몇 번째에 해당하는 단어까지 사용할 것인지 조절

In [ ]:
print(f"훈련용 뉴스 기사 : {len(X_train)}")
print(f"테스트용 뉴스 기사 : {len(X_test)}")
print(f"카테고리 개수 : {max(y_train) + 1}")

In [ ]:
#첫번째 훈련용 뉴스 기사 및 레이블
print(X_train[0])
print(y_train[0])
#X_train의 정수인코딩 => 0: 빈도수가 가장 높은 것, 값이 클수록 빈도수가 낮은 것
#y_train = 3 : 카테고리 3번

In [ ]:
#전체 뉴스 기사
print(f"뉴스 기사의 최대 길이 : {max(len(l) for l in X_train)}")
print(f"뉴스 기사의 평균 길이 : {(sum(len(l) for l in X_train)) / len(X_train)}")

plt.hist([len(l) for l in X_train],bins = 50)
plt.xlabel("length of samples")
plt.ylabel("number of samples")
plt.show()

In [ ]:
fig,ax = plt.subplots(ncols = 1)
fig.set_size_inches(12,5)
sns.countplot(y_train)

In [ ]:
unique_elements, counts_elements = np.unique(y_train, return_counts=True)
print("각 레이블에 대한 빈도수")
print(np.asarray([unique_elements,counts_elements]))
# np.asarray vs np.array
# 기본적으로 두 함수는 같은 기능을 수행
# 차이 : copy()옵션 default : array : True / asarray : False
# 단 type을 convert할 경우(list => array) copy() 옵션이 무시되어 둘 다 동일함

In [ ]:
word_to_index = reuters.get_word_index() #reuters dataset에 인덱스별 카운트 횟수가 포함되어 있음
print(word_to_index)

In [ ]:
#상기와 같이 출력할 경우 어떤 단어에 어떤 인덱스가 부여되었는지 확인하기 어려우므로 좀 더 쉽게 단어를 확인할 수 있도록 변경
index_to_word = {}
for key,value in word_to_index.items():
  index_to_word[value] = key

In [ ]:
print(index_to_word)

In [ ]:
print("빈도수 상위 1번 단어 : ",index_to_word[1])
print("빈도수 상위 28842번 단어 : ",index_to_word[28842])

#보통 불용어로 처리되는 the가 어김없이 등장 빈도수 1위를 차지하는 것을 확인

In [ ]:
# 훈련용 뉴기 기사 중 첫번째 기사가 어떤 구성으로 되어있는지 확인
for index, token in enumerate(("<pad>","<sos>","<unk>")):
  index_to_word[index]=token

print(' '.join([index_to_word[index] for index in X_train[0]]))

## LSTM으로 로이터 뉴스 분류하기

In [ ]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import  EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
(X_train, y_train),(X_test, y_test) = reuters.load_data(num_words=1000, test_split=0.2)

In [ ]:
max_len = 100
X_train = pad_sequences(sequences=X_train, maxlen = max_len)
X_test = pad_sequences(sequences=X_test, maxlen = max_len)

In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
model = Sequential()
model.add(Embedding(1000, 120))
model.add(LSTM(units = 120))
model.add(Dense(46, activation='softmax'))

In [ ]:
#검증 데이터 손실(val_loss)가 증가 시 과적합 징후이므로 검증 데이터 손실이 4회 이상 증가하면 학습을 조기 종료
#또한, ModelCheckPoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델을 저장
es = EarlyStopping(monitor = 'val_loss', mode = "min",verbose = 1, patience = 4)
mc = ModelCheckpoint(filepath = "best_model.h5", monitor='val_acc',mode = 'max', verbose = 1, save_best_only=True)


In [ ]:
model.compile(loss = "categorical_crossentropy", optimizer='adam', metrics = ['acc'])

In [ ]:
history = model.fit(x = X_train, y = y_train, batch_size=128, epochs = 30, callbacks = [es,mc], validation_data=(X_test, y_test))

In [ ]:
loaded_model = load_model("best_model.h5")
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test,y_test)[1]))

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()